DEFINICION DE FUNCIONES

In [2]:
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

# ruta donde se almacenarán las imágenes obtenidas para el entreno
data_dir = 'Images_class_model'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

if os.path.exists(data_dir):
    print(f"El directorio '{data_dir}' se ha creado correctamente.")
else:
    print(f"Error: No se pudo crear el directorio '{data_dir}'.")

def image_capture(class_model_name):
    persona_dir = os.path.join(data_dir, class_model_name)
    if not os.path.exists(persona_dir):
        os.makedirs(persona_dir)

    video_capture = cv2.VideoCapture(0)
    count = 0

    while True:
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            roi = gray[y:y+h, x:x+w]
            
            # Guarda la imagen en la carpeta específica de la persona
            cv2.imwrite(f'{persona_dir}/{class_model_name}_{count}.png', roi)
            count += 1

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q') or count >= 150:
            break

    video_capture.release()
    cv2.destroyAllWindows()

def train_model(data_dir):
    data = []
    labels = []

    for person in os.listdir(data_dir):
        if person.startswith('.'):
            continue

        label = person
        person_dir = os.path.join(data_dir, person)

        for file in os.listdir(person_dir):
            if file.endswith('.png'):
                img_path = os.path.join(person_dir, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (100, 100))
                data.append(np.ravel(img))
                labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    # Hay al menos dos clases para entrenar el clasificador SVM
    if len(np.unique(labels)) > 1:
        X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

        n_components = 50
        pca = PCA(n_components=n_components, whiten=True).fit(X_train)
        X_train_pca = pca.transform(X_train)
        X_test_pca = pca.transform(X_test)

        classifier = SVC(kernel='rbf', class_weight='balanced')
        classifier.fit(X_train_pca, y_train)

        y_pred = classifier.predict(X_test_pca)

        accuracy = accuracy_score(y_test, y_pred)
        print(f'Precisión del modelo: {accuracy}')

        return classifier, pca
    else:
        print("Error: El uso del clasificador SVM requiere de almenos 2 clases.")
        return None, None

def real_time_recognition(classifier, pca):
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        for (x, y, w, h) in faces:
            roi = gray[y:y+h, x:x+w]
            roi = cv2.resize(roi, (100, 100))


            roi_pca = pca.transform(np.ravel(roi).reshape(1, -1))


            label = classifier.predict(roi_pca)[0]

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f'Model: {label}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

El directorio 'Images_class_model' se ha creado correctamente.


CREADOR DE MODELOS

In [3]:
# Capturar imágenes para el dataset
name = 'Modesto'
image_capture(name)

2023-12-13 12:30:13.262 Python[10822:379710] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [4]:
# Entrenar el modelo y realizar el reconocimiento facial en tiempo real
data_dir = 'Images_class_model'
classifier, pca = train_model(data_dir)
real_time_recognition(classifier, pca)

Precisión del modelo: 1.0


KeyboardInterrupt: 